In [5]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-85cdfed1-f780-cfd0-4508-aa1893567920)


In [6]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
%cd drive/My Drive/dataset/standford_dog_breed_dataset/images

/content/drive/My Drive/dataset/standford_dog_breed_dataset/images


In [8]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.datasets import mnist
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, AveragePooling2D, Flatten, Input, Dropout, BatchNormalization
from imutils import paths
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
import os
import numpy as np
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split

In [9]:
# construct the training image generator for data augmentation
aug = ImageDataGenerator(
	rotation_range=20,
	zoom_range=0.15,
	width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.15,
	horizontal_flip=True,
	fill_mode="nearest")

In [10]:
baseModel = MobileNetV2(include_top=False, input_shape=(224, 224, 3))
                      #input_tensor=Input(shape=(224, 224, 3)))


94773248/94765736 [==============================] - 0s 0us/step


In [11]:
# loop over all layers in the base model and freeze them so they will NOT be updated during the first training process
for layer in baseModel.layers:
	layer.trainable = False

In [12]:
# construct the head of the model that will be placed on top of the
# the base model
headModel = baseModel.output
#headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = GlobalAveragePooling2D()(headModel)
headModel = BatchNormalization()(headModel)
#headModel = Flatten(name="flatten")(headModel)
#headModel = Dense(1024, activation="relu")(headModel)
#headModel = Dropout(0.5)(headModel)
headModel = Dense(120, activation="softmax")(headModel)


# place the head FC model on top of the base model (this will become the actual model we will train)
model = Model(inputs=baseModel.input, outputs=headModel)

In [13]:
INIT_LR = 0.0001
EPOCHS = 100
BS = 256
opt = Adam(learning_rate=INIT_LR)
           #decay_step=1000, decay_rate=0.9)
model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])

In [14]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
_______________________________________________________________________________________

In [15]:
H = model.fit(aug.flow_from_directory("Images", batch_size=BS,target_size=(224, 224)), epochs=EPOCHS, verbose=1)

Found 20254 images belonging to 120 classes.
Epoch 1/100
80/80 [==============================] - 7042s 88s/step - loss: 4.7975 - accuracy: 0.0480
Epoch 2/100
80/80 [==============================] - 275s 3s/step - loss: 3.5766 - accuracy: 0.1813
Epoch 3/100
80/80 [==============================] - 274s 3s/step - loss: 2.9030 - accuracy: 0.3017
Epoch 4/100
80/80 [==============================] - 274s 3s/step - loss: 2.5063 - accuracy: 0.3797
Epoch 5/100
80/80 [==============================] - 273s 3s/step - loss: 2.2463 - accuracy: 0.4306
Epoch 6/100
80/80 [==============================] - 273s 3s/step - loss: 2.0633 - accuracy: 0.4687
Epoch 7/100
80/80 [==============================] - 273s 3s/step - loss: 1.9210 - accuracy: 0.4977
Epoch 8/100
80/80 [==============================] - 273s 3s/step - loss: 1.8105 - accuracy: 0.5237
Epoch 9/100
80/80 [==============================] - 273s 3s/step - loss: 1.7231 - accuracy: 0.5506
Epoch 10/100
80/80 [==============================] -